In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
from mctsrunner import train_mcts
from env import DefaultEnvironment
from mcts import GameTree 
from config import GameConfig
from enums import StartingSplit
from player import init_players

In [2]:
project_root = '/home/justiny/Documents/Projects/PyDominion'
model_dir = os.path.join(project_root, 'models')

In [3]:
tree_name = 'r-r-mcts-10k-score'
rollout_name = 'mlog-10k'
tree_path = os.path.join(model_dir, tree_name)
rollout_path = os.path.join(model_dir, rollout_name)

In [32]:
config = GameConfig(split=StartingSplit.StartingRandomSplit, sandbox=True, num_players=2)
tree = GameTree(train=True)
player = init_players(['UCT'], tree=tree, rollout_type='mlog')[0]
players = [player, player]
env = DefaultEnvironment(config, players)

In [71]:
train_mcts(env=env, tree=tree, epochs=5, train_epochs_interval=1, save_epochs=1000, path=tree_path, rollout_path=rollout_path)

100%|██████████| 5/5 [00:00<00:00, 18.01it/s]


In [61]:
i = 0
model = player.rollout.models[i]

In [77]:
from rollout import LogisticRegressionEnsembleRollout
import torch

In [75]:
model = LogisticRegressionEnsembleRollout()

In [78]:
data = {'features': np.array([[1,2], [3,4]]), 'idxs': {'Province': 0}, 'rewards': [1,2]}

In [83]:
model.update(**data)

ValueError: only one element tensors can be converted to Python scalars